In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import re
import ast


In [32]:
# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
# tokenizer.save_pretrained("../backend/models/Qwen/tokenizer")
# model.save_pretrained("../backend/models/Qwen/model")

tokenizer = AutoTokenizer.from_pretrained("../backend/models/Qwen/tokenizer")
model = AutoModelForCausalLM.from_pretrained("../backend/models/Qwen/model")

Loading weights: 100%|██████████| 339/339 [00:00<00:00, 1138.28it/s, Materializing param=model.norm.weight]                              


In [33]:
def translate_with_llm(text):
    messages = [
    {
        "role": "system", 
        "content": """
            You are a professional Manga Localizer. You will receive JSON-formatted Japanese OCR text from the manga "Naruto."

            CRITICAL INSTRUCTIONS:

            OUTPUT ONLY ENGLISH: Do not provide Romaji or Japanese in the final translation.

            FIX OCR ERRORS: The input has errors (e.g., 'バー' might be 'バカ'). Use your knowledge of Naruto to correct them.

            CONVINCING DIALOGUE: Use character-specific voices (e.g., Naruto's "Believe it!", Iruka's stern teacher tone).

            MATCH IDs: Return the response as a JSON object matching the input IDs.

        """
    },
    {
        "role": "user", 
        "content": text
    },
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    outputs = model.generate(**inputs, max_new_tokens=400)
    output_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    return output_text

In [ ]:
def parse_and_clean(json_str):
    try:
        # 1. Handle cases where the LLM adds "Here is the JSON:" text
        match = re.search(r'\[.*\]', json_str, re.DOTALL)
        if match:
            clean_str = match.group(0)
            data = ast.literal_eval(clean_str)

        for item in data:
            idx = item.get('id', 0)
            text = item.get('text', '').replace('\n', ' ').strip()
            
            print(f"{idx}: {text}")
            
    except Exception as e:
        print(f"Failed to parse JSON: {e}")

In [40]:
result = translate_with_llm("""[
    {'id': 0, 'text': 'バーうっせんだってばよ！！'}, 
    {'id': 1, 'text': 'お前ちさ！お前らさ！こんな卑劣なことできねーだろ！！'}, 
    {'id': 2, 'text': 'だがォレはできる！！オレはスゴイ！！'},
    {'id': 3, 'text': 'おーおー ！やってくれとるのォあのバカ！'}, 
    {'id': 4, 'text': 'ん？'}, {'id': 5, 'text': 'ん？'}, 
    {'id': 6, 'text': '三代目申し訳ありません！'}, 
    {'id': 7, 'text': 'お！イルカか．．．'}, 
    {'id': 8, 'text': 'やべ！イルカ先生 だ'}, 
    {'id': 9, 'text': '何やってんだ授業中だぞ！早く降りてこい！！'}
    ]
""")
parse_and_clean(result)

Failed to parse JSON: Extra data: line 1 column 51 (char 50)


In [45]:
parse_and_clean(result)

Bubble 0: Hey shut up already damn it!!
Bubble 1: You brat! You guys! You can't do something so lowly!!
Bubble 2: But I can do it!! I'm awesome!!
Bubble 3: Oh oh! That's Naruto! That idiot is actually doing it!
Bubble 4: Hm?
Bubble 5: Hm?
Bubble 6: Third Hokage, I apologize!
Bubble 7: Ah! It's Iruka...
Bubble 8: Shit! It's Iruka-sensei!
Bubble 9: What are you doing? This is class time! Get down here right now!!
